In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import dateutil
import sys
import logging
from typing import List
from google.cloud import bigquery
from google.cloud import bigquery
from google.oauth2 import service_account

In [2]:
#-------------------Extract-------------------------
urls = ["https://forecast.weather.gov/MapClick.php?lat=57.0826&lon=-135.2692#.Y-vs_9LMJkg",
        'https://forecast.weather.gov/MapClick.php?lat=45.5118&lon=-122.6756#.Y-vtHNLMJkg',
        ]
        
combined_df = pd.DataFrame()

for url in urls:
    r = requests.get(url)
    soup = BeautifulSoup(r.content,"html.parser")

    #various containers
    item1 = soup.find_all(id='current_conditions-summary')
    item2 = soup.find_all(id='current_conditions_detail')
    item4 = soup.find_all(id='tombstone-container')

    #raw data
    temp_f = [item.find(class_="myforecast-current-lrg").get_text() for item in item1]
    temp_min = soup.find('p', {'class': 'temp temp-low'}).text.strip()
    temp_max = soup.find('p', {'class': 'temp temp-high'}).text.strip()


    #df of temperatures
    df_temperature = pd.DataFrame({"temp" : temp_f,'tempmin': temp_min,'tempmax': temp_max})

    #df_2 is a df of current conditions in detail (Humidity, Wind Speed, Barometer, Dewpoint, Visibility, Last update)
    table = soup.find_all('table')
    df_2 = pd.read_html(str(table))[0]
    df_2 = df_2.pivot(columns=0, values=1).ffill().dropna().reset_index().drop(columns=['index'])

    #merge both dataframes
    temp_df=pd.concat([df_temperature,df_2],axis=1)

    #scrape lattitude, longitude, and elevation 
    lat_lon_elev = soup.find('span', {'class': 'smallTxt'}).text.strip()
    lat, lon, elev = re.findall(r'[-+]?\d*\.\d+|\d+', lat_lon_elev)

    #scrape name
    station = soup.find('h2', {'class': 'panel-title'}).text.strip()

    #add location, lat, long, and elev to source_df
    temp_df['elevation_ft'] = elev
    temp_df['latitude'] = lat
    temp_df['longitude'] = lon
    temp_df['weather_station'] = station

    combined_df = pd.concat([temp_df, combined_df], ignore_index=True, sort=False)

display(combined_df)



,temp,tempmin,tempmax,Barometer,Dewpoint,Humidity,Last update,Visibility,Wind Chill,Wind Speed,elevation_ft,latitude,longitude,weather_station
0,43°F,Low: 33 °F,High: 46 °F,29.78 in (1008.47 mb),36°F (2°C),75%,21 Feb 08:35 AM PST,10.00 mi,38°F (3°C),SSW 9 MPH,20,45.59578,122.60917,"Portland, Portland International Airport (KPDX)"
1,34°F,Low: 18 °F,High: 35 °F,30.26 in (1024.8 mb),5°F (-15°C),29%,21 Feb 7:53 am AKST,10.00 mi,26°F (-3°C),NE 10 mph,13,57.05,135.36,Sitka - Sitka Airport (PASI)


In [34]:
urls = ['https://www.localconditions.com/weather-portland-oregon/97201/past.php',
        "https://www.localconditions.com/weather-sitka-alaska/99835/past.php",
        ]

precip_df = pd.DataFrame()

for url in urls:
        r = requests.get(url)
        soup = BeautifulSoup(r.content,"html.parser")
        details = soup.select_one(".past_weather_express")
        # Find all div elements with class="panel"
        panel_divs = soup.find_all('div', {'class': 'panel'})
        # Extract the text content of each div element and store it in a list
        panel_texts = [panel_div.text.strip() for panel_div in panel_divs]
        # Print the list of extracted text content
        data = panel_texts[1]
        data = [item.strip() for item in data]
        data = [item for item in data if item]
        data = data[79:90]
        data="".join(data)
        df = pd.DataFrame([data], columns=['precip'])
        precip_df = pd.concat([precip_df, df], ignore_index=True, sort=False)
        
precip_df['precip'] = precip_df['precip'].str.extract(pat='(\d+\.?\d*)').astype(float)
precip_df = precip_df.fillna(0)

source_df =pd.concat([combined_df,precip_df],axis=1)
display(source_df)

,temp,tempmin,tempmax,Barometer,Dewpoint,Humidity,Last update,Visibility,Wind Chill,Wind Speed,elevation_ft,latitude,longitude,weather_station,precip
0,43°F,Low: 33 °F,High: 46 °F,29.78 in (1008.47 mb),36°F (2°C),75%,21 Feb 08:35 AM PST,10.00 mi,38°F (3°C),SSW 9 MPH,20,45.59578,122.60917,"Portland, Portland International Airport (KPDX)",0.080
1,34°F,Low: 18 °F,High: 35 °F,30.26 in (1024.8 mb),5°F (-15°C),29%,21 Feb 7:53 am AKST,10.00 mi,26°F (-3°C),NE 10 mph,13,57.05,135.36,Sitka - Sitka Airport (PASI),0.077


In [4]:
#-----------Data Transformations-----------------

# Convert 'lat' and 'lon' columns to float type
source_df['latitude'] = source_df['latitude'].astype(float)
source_df['longitude'] = source_df['longitude'].astype(float)

# Convert 'elev' column to int type
source_df['elevation_ft'] = source_df['elevation_ft'].astype(int)

# Extract the numeric part of the temperature string and convert it to int
source_df['temp'] = source_df['temp'].str.extract('(\d+)').astype(float)

# Extract the numeric part of the tempmin string and convert it to int
source_df['tempmin'] = source_df['tempmin'].str.extract('(\d+)').astype(float)

# Extract the numeric part of the temperature string and convert it to int
source_df['tempmax'] = source_df['tempmax'].str.extract('(\d+)').astype(float)

# Split wind speed values into components and convert speed to int type
source_df['Wind Speed'] = source_df['Wind Speed'].str.extract('(\d+)', expand=False).fillna(0).astype(float)

# Convert 'humidity' column to int type
source_df['Humidity'] = source_df['Humidity'].str.extract('(\d+)', expand=False).astype(float)

# Convert 'barometer' column to float type, and convert inches to millibars
source_df['Barometer'] = round(source_df['Barometer'].apply(lambda x: float(x.split()[0]) * 33.8639 if 'in' in x and x != 'NA' else None), 2)

# Convert 'Visibility' column to float type
source_df['Visibility'] = source_df['Visibility'].str.extract('(\d+\.\d+|\d+)', expand=False).astype(float).round(2)

#Convert 'last_update' column to UTC
source_df['Last update'] = source_df['Last update'].apply(lambda x: dateutil.parser.parse(x, tzinfos={"EST": -5 * 3600, "CST": -6 * 3600, "MST": -7 * 3600,"PST": -8 * 3600,"AKST": -9 * 3600,"HST": -10 * 3600}))
source_df['Last update'] = source_df['Last update'].apply(lambda x: x.astimezone(dateutil.tz.tzutc()))
source_df['datetime'] = source_df['Last update'].dt.strftime('%Y-%m-%d')
source_df['datetime'] = pd.to_datetime(source_df['datetime'])

# make wind chill a float if exists and only display degree F
try:
    source_df[['Wind Chill']] = source_df['Wind Chill'].str.extract('(\d+)', expand=True).astype(float)
except:
    None

# extract the numeric value of dewpoint and only display the degree n farenheit
source_df[['Dewpoint']] = source_df['Dewpoint'].str.extract('(\d+)', expand=True).astype(float)

#change precip data type to float
source_df['precip'] = source_df['precip'].astype(float)

#rename weather station column to the city
def rename_station(value):
    if value == 'Portland, Portland International Airport (KPDX)':
        return 'Portland'
    elif value == 'Sitka - Sitka Airport (PASI)':
        return 'Sitka'

source_df['name'] = source_df['weather_station'].map(rename_station)

#change the names and order of columns to better fit the historical data
source_df = source_df.rename({'Humidity': 'humidity', 'Wind Speed': 'windspeed', 'Visibility': 'visibility','Wind Chill': 'windchill','Dewpoint':'dewpoint'}, axis=1) 
#this line only includes necesarry columns
source_df = source_df.reindex(['name','datetime','tempmax','tempmin','temp','windchill','dewpoint','humidity','precip','windspeed','visibility'], axis=1)
source_df = source_df.fillna(0)



display(source_df)

,name,datetime,tempmax,tempmin,temp,windchill,dewpoint,humidity,precip,windspeed,visibility
0,Portland,2023-02-21,46.0,33.0,43.0,38.0,36.0,75.0,0.003,9.0,10.0
1,Sitka,2023-02-21,35.0,18.0,34.0,26.0,5.0,29.0,0.000,10.0,10.0


In [5]:
#------------------load to BigQuery---------------------------------
#authorization
key_path = "/home/reed/.creds/dsa-deb-sa.json"
credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"])

client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

In [6]:
# **** SETUP LOGGING ****
# setup logging and logger
logging.basicConfig(            # setting up the root logger
    format='[%(levelname)-5s][%(asctime)s][%(module)s:%(lineno)04d] : %(message)s',
    level=logging.INFO,
    stream=sys.stdout
)
logger: logging.Logger = logging.getLogger('root')      # alias the root logger as `logger`
logger.setLevel(logging.DEBUG)                          # programmatically reassign the logging level

In [7]:
# change to match your filesystem
PROJECT_NAME = credentials.project_id
DATASET_NAME = "reed_weather_data"

# **** BIGQUERY DATASET CREATION ****

dataset_id = f"{PROJECT_NAME}.{DATASET_NAME}"
dataset = bigquery.Dataset(dataset_id)
dataset.location = "US"
dataset = client.create_dataset(dataset, exists_ok=True)

logger.info(f"Created weather dataset: {dataset.full_dataset_id}")

FACTS_TABLE_METADATA = {
    'weather_data': {
        'table_name': 'weather_data',
        'schema': [
            # indexes are written if only named in the schema
            bigquery.SchemaField('name', 'STRING', mode='REQUIRED'),
            bigquery.SchemaField('datetime', 'DATETIME', mode='NULLABLE'),
            bigquery.SchemaField('tempmax', 'FLOAT64', mode='NULLABLE'),
            bigquery.SchemaField('tempmin', 'FLOAT64', mode='NULLABLE'),
            bigquery.SchemaField('temp', 'FLOAT64', mode='NULLABLE'),
            bigquery.SchemaField('windchill', 'FLOAT64', mode='NULLABLE'),
            bigquery.SchemaField('dewpoint', 'FLOAT64', mode='NULLABLE'),
            bigquery.SchemaField('humidity', 'FLOAT64', mode='NULLABLE'),
            bigquery.SchemaField('precip', 'FLOAT64', mode='NULLABLE'),
            bigquery.SchemaField('windspeed', 'FLOAT64', mode='NULLABLE'),
            bigquery.SchemaField('visibility', 'FLOAT64', mode='NULLABLE'),
        ]
    }      
}

[INFO ][2023-02-21 09:16:26,157][3414794505:0012] : Created weather dataset: deb-01-372116:reed_weather_data


In [8]:
def load_table(
    df: pd.DataFrame, 
    client: bigquery.Client, 
    table_name: str, 
    schema: List[bigquery.SchemaField], 
    create_disposition: str = 'CREATE_IF_NEEDED', 
    write_disposition: str = 'WRITE_TRUNCATE'
    ) -> None:
    """load dataframe into bigquery table

    Args:
        df (pd.DataFrame): dataframe to load
        client (bigquery.Client): bigquery client
        table_name (str): full table name including project and dataset id
        schema (List[bigquery.SchemaField]): table schema with data types
        create_disposition (str, optional): create table disposition. Defaults to 'CREATE_IF_NEEDED'.
        write_disposition (str, optional): overwrite table disposition. Defaults to 'WRITE_TRUNCATE'.
    """
    # *** run some checks ***
    # test table name to be full table name including project and dataset name. It must contain to dots
    assert len(table_name.split('.')) == 3, f"Table name must be a full bigquery table name including project and dataset id: '{table_name}'"
    # setup bigquery load job:
    #  create table if needed, replace rows, define the table schema
    job_config = bigquery.LoadJobConfig(
        create_disposition=create_disposition,
        write_disposition=write_disposition,
        schema=schema
    )
    logger.info(f"loading table: '{table_name}'")
    job = client.load_table_from_dataframe(df, destination=table_name, job_config=job_config)
    job.result()        # wait for the job to finish
    # get the resulting table
    table = client.get_table(table_name)
    logger.info(f"loaded {table.num_rows} rows into {table.full_table_id}")

In [9]:
# Load scraped Sitka and Portland daily weather to the historical data table

# get table name and schema from FACTS_TABLE_METADATA config param
table_name = f"{PROJECT_NAME}.{DATASET_NAME}.{FACTS_TABLE_METADATA['weather_data']['table_name']}"
schema = FACTS_TABLE_METADATA['weather_data']['schema']
# load dataframe
load_table(source_df, client, table_name, schema)

logger.info(f"loaded weather_data")

[INFO ][2023-02-21 09:16:26,264][1947479415:0029] : loading table: 'deb-01-372116.reed_weather_data.weather_data'
[INFO ][2023-02-21 09:16:31,304][1947479415:0034] : loaded 2 rows into deb-01-372116:reed_weather_data.weather_data
[INFO ][2023-02-21 09:16:31,305][3042518909:0009] : loaded weather_data


In [10]:
#-------------------------Working Selinium code but does not work with airflow. an alternate method is being used---------------------

# from selenium import webdriver
# from time import sleep
# from selenium.webdriver.common.by import By

# URLS = ["https://www.wunderground.com/history/daily/us/or/portland",
#         'https://www.wunderground.com/history/daily/us/ak/sitka/PASI'
#         ]

# precip_df = pd.DataFrame()

# for URL in URLS:
    
#     #create selenium web driver
#     driver = webdriver.Chrome(r"C:/home/reed/.cache/selenium/chromedriver/linux64/109.0.5414.74")
#     driver.get(URL)
#     sleep(5)

#     soup = BeautifulSoup(driver.page_source, "html.parser")
    
#     #closes the pop up add if it appears otherwise continues to scrape the precip value from the page
#     try:
#         driver.find_element(By.XPATH,'/html/body/div[9]/div[3]/div/div/div/a').click()
#     except:
#         precip = driver.find_element(By.XPATH,"/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div/section/div[2]/div[1]/div[3]/div[1]/div/lib-city-history-summary/div/div[2]/table/tbody[2]/tr/td[1]").text

#     df = pd.DataFrame([precip], columns=['precip'])
#     precip_df = pd.concat([precip_df, df], ignore_index=True, sort=False)

#     #quits the selenium driver
#     driver.quit()

# source_df =pd.concat([combined_df,precip_df],axis=1)
# display(source_df)